In [1]:
import tensorflow as tf

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pickle
import random
import numpy as np


In [3]:
#set parameters
image_size=32
num_channels=3
num_categories=10
num_filters=32
filter_size=5
num_epochs=200
batch_size=10
num_batches=int(50000/batch_size)
keep_prob=0.6

In [4]:
#Read CIFAR training data
train_data=None
train_labels=None
for file_index in range(5):
    train_file=open("data_batch_1",'rb')
    train_dict=pickle.load(train_file,encoding='latin1')
    train_file.close()
    
    if train_data is None:
        train_data=np.array(train_dict['data'],float)/255.0
        train_labels=train_dict["labels"]
    else:
        train_data=np.concatenate((train_data,train_dict['data']),0)
        train_labels=np.concatenate((train_labels,train_dict['labels']),0)



''' Serialization means to convert an object into that string, and 
deserialization is its inverse operation. When transmitting data or storing them in a file, 
the data are required to be byte strings, but complex objects are seldom in this format.'''

'''Pickle is the standard way of serializing objects in Python.
You can use the pickle operation to serialize your machine learning algorithms and 
save the serialized format to a file. Later you can load this file to deserialize your model 
and use it to make new predictions.'''
    
    
'''Join a sequence of arrays along an existing axis.----concatenate'''



'Join a sequence of arrays along an existing axis.----concatenate'

In [5]:
#preprocess training data and labels
train_data=train_data.reshape([-1,num_channels,image_size,image_size])
train_data=train_data.transpose([0,2,3,1])
train_labels=np.eye(num_categories)[train_labels]

'''numpy.eye(R, C = None, k = 0, dtype = type <‘float’>) : 
#–The eye tool returns a 2-D array with  1’s as the diagonal and  0’s elsewhere. Matrix b : 
# [[ 1.  0.]
#[ 0.  1.]]'''

'''Matrix a : 
 [[ 0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]] '''



'Matrix a : \n [[ 0.  0.  0.  0.  0.]\n [ 1.  0.  0.  0.  0.]\n [ 0.  1.  0.  0.  0.]\n [ 0.  0.  1.  0.  0.]] '

In [6]:
#read  CIFAR test data
test_file=open("data_batch_1",'rb')
test_dict=pickle.load(test_file,encoding='latin1')
test_file.close()
test_data=test_dict['data']
test_labels=test_dict['labels']



In [7]:
#preprocess test data and labels
test_data=test_data.reshape([-1,num_channels,image_size,image_size])
test_data=test_data.transpose([0,2,3,1])
test_labels=np.eye(num_categories)[test_labels]



# TensorFlow Variables:

--> When we train the model, we need to assign some weights and biases throughout the session.

--> TensorFlow variables can hold the values of biases and weights throughout the session.

--> You need to one thing keep in mind thatTensorFlow variables need to be initialized.

--> In TensorFlow variables are of great use when we are training models. As constants, we have to call a constructor to initialize a variable; the initial value can be passed in as an argument.

--> Variables can easily be added to the computational graph by calling a constructor.

# TensorFlow Placeholder:

--> TensorFlow placeholders are initially empty and are used to feed in the actual training examples.
--> If we want to inject the data into a computation graph, we have to use the mechanism named as a placeholder.      
    Placeholders are bound inside some expressions. The syntax of the placeholder is following.
  # placeholder(dtype, shape=None, name=None)
--> TensorFlow Placeholder does need to declare as a float32 datatype within an optional shape parameter.
""" Placeholders allow us to not to provide the data in advance for operations and computation graphs, and the data can be added in runtime from external sources as we train the Machine Learning models."""

In [8]:
#placeholders for CIFAR images
img_holder=tf.placeholder(tf.float32,[None,image_size,image_size,num_channels])
lbl_holder=tf.placeholder(tf.float32,[None,num_categories])
train=tf.placeholder(tf.bool) #placeholder for a single boolean value

    


AttributeError: module 'tensorflow' has no attribute 'placeholder'

# Intro to Convolutional Neural Networks
    Convolutional neural networks (CNNs):
        CNNs are the current state-of-the-art model architecture for image classification tasks. CNNs apply a series of filters to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains three components:

    Convolutional layers:
        which apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. Convolutional layers then typically apply a ReLU activation function to the output to introduce nonlinearities into the model.

    Pooling layers:
        which downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values.

    Dense (fully connected) layers:
        which perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.

 # softmax activation function :
     To generate a value between 0–1 for each node (the sum of all these softmax values is equal to 1). We can interpret the softmax values for a given image as relative measurements of how likely it is that the image falls into each target class.

#  CNN architecture:

    Convolutional Layer #1: 
       Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function
    Pooling Layer #1:
        Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)
    Convolutional Layer #2: 
        Applies 64 5x5 filters, with ReLU activation function
    Pooling Layer #2:
        Again, performs max pooling with a 2x2 filter and stride of 2
    Dense Layer #1:
        1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training)
    Dense Layer #2 (Logits Layer): 
        10 neurons, one for each digit target class (0–9).

# The tf layers module contains methods to create each of the three layer types above:

    conv2d():
        Constructs a two-dimensional convolutional layer. Takes number of filters, filter kernel size, padding, and activation function as arguments.
    max_pooling2d():
        Constructs a two-dimensional pooling layer using the max-pooling algorithm. Takes pooling filter size and stride as arguments.
    dense():
    Constructs a dense layer. Takes number of neurons and activation function as arguments.
    Dropout():
    It is useful for regularizing DNN models. Inputs elements are randomly set to zero (and the other elements are rescaled). This encourages each node to be independently useful, as it cannot rely on the output of other nodes.

In [9]:
#create convolution/pooling layers
conv1 = tf.layers.conv2d(img_holder,num_filters,filter_size,padding = "same",activation = tf.nn.relu)
drop1 = tf.layers.dropout(conv1,keep_prob,training = train)
pool1 = tf.layers.max_pooling2d(drop1,2,2)
conv2 = tf.layers.conv2d(pool1,num_filters,filter_size,padding = "same",activation = tf.nn.relu)
drop2 = tf.layers.dropout(conv2,keep_prob,training = train)
pool2 = tf.layers.max_pooling2d(drop2,2,2)
conv3 = tf.layers.conv2d(pool2,num_filters,filter_size,padding = "same",activation = tf.nn.relu)
pool3 = tf.layers.max_pooling2d(conv3,2,2)
conv4 = tf.layers.conv2d(pool3,num_filters,filter_size,padding = "same",activation = tf.nn.relu)
drop3 = tf.layers.dropout(conv4,keep_prob,training = train)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


In [10]:
#flatten input data
flatten=tf.reshape(drop3,[-1,512])



     tf.contrib.layers.fully_connected  :
    creates a variable called weights, representing a fully connected weight matrix, which is multiplied by the inputs to produce a Tensor of hidden units. 
    If a ""normalizer_fn"" is provided (such as batch_norm), it is then applied. Otherwise, if normalizer_fn is None and a biases_initializer is provided then a biases variable would be created and added the hidden units. Finally, if activation_fn is not None, it is applied to the hidden units as well.
    Normalizer_params: 
        Normalization function parameters.


    #inputs: 
    A tensor of at least rank 2 and static value for the last dimension; i.e. [batch_size, depth], [None, None, None, channels].
    #num_outputs: 
    Integer or long, the number of output units in the layer.
    #activation_fn: 
    Activation function. The default value is a ReLU function. Explicitly set it to None to skip it and maintain a linear activation.
    #normalizer_fn: 
    Normalization function to use instead of biases. If normalizer_fn is provided then biases_initializer and biases_regularizer are ignored and biases are not created nor added. default set to None for no normalizer function
    normalizer_params: Normalization function parameters.
    weights_initializer: An initializer for the weights.
    weights_regularizer: Optional regularizer for the weights.
    biases_initializer: An initializer for the biases. If None skip biases.
    biases_regularizer: Optional regularizer for the biases.
    reuse:
        Whether or not the layer and its variables should be reused. To be able to reuse the layer scope must be given.
    variables_collections: 
        Optional list of collections for all the variables or a dictionary containing a different list of collections per variable.
    outputs_collections: Collection to add the outputs.
    trainable:
    If True also add variables to the graph collection GraphKeys.TRAINABLE_VARIABLES (see tf.Variable).
    scope: Optional scope for variable_scope.

In [11]:
#create connected layers
# tf.contrib.framework.arg_scope: Stores the default arguments for the given set of list_ops.


with tf.contrib.framework.arg_scope([tf.contrib.layers.fully_connected],
                                   normalizer_fn=tf.contrib.layers.batch_norm,
                                   normalizer_params={'is_training':train}):
    fc1=tf.contrib.layers.fully_connected(flatten,512)
    fc2=tf.contrib.layers.fully_connected(fc1,num_categories,activation_fn=None)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



    Logits: simply means that the function operates on the unscaled output of earlier layers and that the relative scale to understand the units is linear. It means, in particular, the sum of the inputs may not equal 1, that the values are not probabilities (you might have an input of 5).

    tf.nn.softmax: produces just the result of applying the softmax function to an input tensor. The softmax "squishes" the inputs so that sum(input) = 1: it's a way of normalizing. The shape of output of a softmax is the same as the input: it just normalizes the values. The outputs of softmax can be interpreted as probabilities.
    tf.nn.softmax_cross_entropy_with_logits: computes the cross entropy of the result after applying the softmax function (but it does it all together in a more mathematically careful way).

In [12]:
#compute loss
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc2,labels=lbl_holder))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
#create optimizer
learning_rate=0.0005
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss)



In [14]:
#initialize variables
init=tf.global_variables_initializer()



In [15]:
#launch session
with tf.Session() as sess:
    sess.run(init)
    #loop over epochs
    for epoch in range(num_epochs):
        #loop over batches
        for batch in range(num_batches):
            batch_start=random.randint(0,batch_size*(num_batches-1)-1)
            batch_end=batch_start+batch_size
            img_batch=train_data[batch_start:batch_end,:]
            lbl_batch=train_labels[batch_start:batch_end,:]
            sess.run(optimizer,feed_dict={img_holder:img_batch,lbl_holder:lbl_batch,train:True})
    #determine success rate
    prediction=tf.equal(tf.argmax(fc2,1),tf.argmax(lbl_holder,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))
    print('Accuracy:',sess.run(accuracy,feed_dict={img_holder:test_data,lbl_holder:test_labels,train:False}))
        


KeyboardInterrupt: 